In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!sudo apt-get update -qq
!sudo apt-get install -qq -y ffmpeg

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [6]:
!pip install -q git+https://github.com/m-bain/whisperx.git
!pip install -q faster-whisper

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.9 MB/

In [16]:
import whisperx
import torch
import gc
import traceback
from google.colab import files, drive
import os
import shutil


In [3]:
language_code = "en"
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "int8" if device == "cpu" else "float16"

In [4]:
audio_dir = "/content/drive/MyDrive/fMRI/stimuli"
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".wav")]

In [13]:
model_a, metadata = whisperx.load_align_model(language_code=language_code, device=device)

In [14]:
for file in audio_files:
    audio_file = os.path.join(audio_dir, file)
    transcript_file = os.path.join(audio_dir, file.replace(".wav", "_transcript.txt"))
    output_tsv_file = os.path.join(audio_dir, file.replace(".wav", "_alignment_output.tsv"))

    print(f"Loading audio from: {audio_file}")
    print(f"Loading transcript from: {transcript_file}")


    try:

      audio = whisperx.load_audio(audio_file)

      audio_duration = audio.shape[0] / 16000.0

      with open(transcript_file, 'r', encoding='utf-8') as f:
          transcript_text = f.read()


    # process the transcript
      result_transcribe = {
          "segments": [{
              "text": transcript_text,
              "start": 0,
              "end": audio_duration
         }]
      }

      # Alignment
      print("Aligning transcription...")
      result_aligned = whisperx.align(
          result_transcribe["segments"],
          model_a,
          metadata,
          audio,
          device,
          return_char_alignments=False
     )

      # Get word segments and confirm alignment
      word_segments = result_aligned.get("word_segments", [])
      if not word_segments:
          print("Alignment failed: No word segments found.")
          print("Full aligned result:", result_aligned)
      else:
         # Write/format and save to .tsv
          print(f"\n--- Alignment Complete ---")
          print(f"Saving word-level timestamps to {output_tsv_file}...")

          with open(output_tsv_file, 'w', encoding='utf-8') as f:

              f.write("word\tstart\tend\n")

              for segment in word_segments:
                  word = segment.get('word', 'N/A').strip()
                  start_time = segment.get('start', 'N/A')
                  end_time = segment.get('end', 'N/A')

                  if isinstance(start_time, float) and isinstance(end_time, float):
                      f.write(f"{word}\t{start_time:.3f}\t{end_time:.3f}\n")
                  else:
                      # Handle cases where timestamps might be missing
                      f.write(f"{word}\t{start_time}\t{end_time}\n")

          print("Successfully saved the output.")

    except Exception as e:
        print(f"\nAn error occurred: {e}")
        print("\n--- Traceback ---")
        traceback.print_exc()
        print("-----------------")

    finally:
        print("\nCleaning up memory...")
        gc.collect()
        torch.cuda.empty_cache()


Loading audio from: /content/drive/MyDrive/fMRI/stimuli/21styear_audio.wav
Loading transcript from: /content/drive/MyDrive/fMRI/stimuli/21styear_audio_transcript.txt
Loading alignment model...
Aligning transcription...

An error occurred: CUDA out of memory. Tried to allocate 20.60 GiB. GPU 0 has a total capacity of 39.56 GiB of which 17.85 GiB is free. Process 24875 has 21.70 GiB memory in use. Of the allocated memory 21.16 GiB is allocated by PyTorch, and 50.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

--- Traceback ---
-----------------

Cleaning up memory...


Traceback (most recent call last):
  File "<ipython-input-14-1eabcf04c28e>", line 35, in <cell line: 0>
    result_aligned = whisperx.align(
                     ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/whisperx/__init__.py", line 16, in align
    return alignment.align(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/whisperx/alignment.py", line 253, in align
    emissions, _ = model(waveform_segment.to(device), lengths=lengths)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3

Loading audio from: /content/drive/MyDrive/fMRI/stimuli/bigbang_audio.wav
Loading transcript from: /content/drive/MyDrive/fMRI/stimuli/bigbang_audio_transcript.txt
Loading alignment model...
Aligning transcription...

--- Alignment Complete ---
Saving word-level timestamps to /content/drive/MyDrive/fMRI/stimuli/bigbang_audio_alignment_output.tsv...
Successfully saved the output.

Cleaning up memory...
Loading audio from: /content/drive/MyDrive/fMRI/stimuli/black_audio.wav
Loading transcript from: /content/drive/MyDrive/fMRI/stimuli/black_audio_transcript.txt
Loading alignment model...
Aligning transcription...

--- Alignment Complete ---
Saving word-level timestamps to /content/drive/MyDrive/fMRI/stimuli/black_audio_alignment_output.tsv...
Successfully saved the output.

Cleaning up memory...
Loading audio from: /content/drive/MyDrive/fMRI/stimuli/bronx_audio.wav
Loading transcript from: /content/drive/MyDrive/fMRI/stimuli/bronx_audio_transcript.txt
Loading alignment model...
Aligning 

In [17]:
audio_dir = "/content/drive/MyDrive/fMRI/stimuli"

audio_recording_dir = os.path.join(audio_dir, "original_audio_recordings")
transcript_dir = os.path.join(audio_dir, "transcripts")
tsv_dir = os.path.join(audio_dir, "force_aligned_perword_timestamp")

os.makedirs(audio_recording_dir, exist_ok=True)
os.makedirs(transcript_dir, exist_ok=True)
os.makedirs(tsv_dir, exist_ok=True)

for file in os.listdir(audio_dir):
    file_path = os.path.join(audio_dir, file)
    if os.path.isfile(file_path):
        if file.endswith(".wav"):
            shutil.move(file_path, os.path.join(audio_recording_dir, file))
        elif file.endswith(".txt"):
            shutil.move(file_path, os.path.join(transcript_dir, file))
        elif file.endswith(".tsv"):
            shutil.move(file_path, os.path.join(tsv_dir, file))

In [19]:
def count_files_in_folder(folder_path):
    return len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

# Count files in each folder
num_audio_files = count_files_in_folder(audio_dir)
num_transcript_files = count_files_in_folder(transcript_dir)
num_tsv_files = count_files_in_folder(tsv_dir)

# Print counts
print(f"Number of .wav files in 'original_audio_recordings': {num_audio_files}")
print(f"Number of .txt files in 'transcripts': {num_transcript_files}")
print(f"Number of .tsv files in 'force_aligned_perword_timestamp': {num_tsv_files}")


Number of .wav files in 'original_audio_recordings': 3
Number of .txt files in 'transcripts': 29
Number of .tsv files in 'force_aligned_perword_timestamp': 28


In [22]:
# List all .txt and .tsv filenames (without extension), cleaned
transcript_files = {os.path.splitext(f.strip())[0].lower() for f in os.listdir(transcript_dir) if f.endswith(".txt")}
tsv_files = {os.path.splitext(f.strip())[0].lower() for f in os.listdir(tsv_dir) if f.endswith(".tsv")}

# Debug print
print(f"Transcript files (count {len(transcript_files)}): {sorted(transcript_files)}")
print(f"TSV files (count {len(tsv_files)}): {sorted(tsv_files)}")



Transcript files (count 29): ['21styear_audio_transcript', 'bigbang_audio_transcript', 'black_audio_transcript', 'bronx_audio_transcript', 'forgot_audio_transcript', 'friends_audio_transcript', 'himym_audio_transcript', 'lucy_audio_transcript', 'merlin_audio_transcript', 'milkywayoriginal_audio_transcript', 'milkywaysynonyms_audio_transcript', 'milkywayvodka_audio_transcript', 'notthefall_audio_transcript', 'notthefallintact_audio_transcript', 'notthefalllongscram_audio_transcript', 'notthefallshortscram_audio_transcript', 'pieman_audio_transcript', 'piemanpni_audio_transcript', 'prettymouth_audio_transcript', 'santa_audio_transcript', 'seinfeld_audio_transcript', 'shame_audio_transcript', 'shapesphysical_audio_transcript', 'shapessocial_audio_transcript', 'sherlock_audio_transcript', 'slumlordreach_audio_transcript', 'tunnel_audio_transcript', 'upintheair_audio_transcript', 'vinny_audio_transcript']
TSV files (count 28): ['bigbang_audio_alignment_output', 'black_audio_alignment_output

In [35]:
tlist = []
tslist = []
for file in transcript_files:
  tlist.append(file[:-11])
for tfile in tsv_files:
  tslist.append(file[:-17])